# GARCH(1,1)

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
from arch import arch_model
import warnings
warnings.filterwarnings('ignore')
import sys
from scipy import stats

In [2]:
snp =pd.read_excel('S&P_BSE.xlsx') 
nif = pd.read_excel('NIFTY.xlsx') 

current_asset_value_snp = 1
units_snp = 1/snp['Close'].iloc[-1]
current_asset_value_nif = 1
units_nif = 1/nif['Close'].iloc[-1]

In [3]:
print("Current value of S&P BSE 500 as on",snp['Date'].iloc[-1],"is",current_asset_value_snp)
print("Current value of NIFTY as on",nif['Date'].iloc[-1],"is",current_asset_value_nif)


Current value of S&P BSE 500 as on 2021-04-09 00:00:00 is 1
Current value of NIFTY as on 2021-04-09 00:00:00 is 1


In [4]:
snp = (snp.set_index('Date'))
nif = (nif.set_index('Date'))
snp = snp['Close']
nif = nif['Close']

In [5]:
returns_snp = ((np.log(snp) - np.log(snp.shift(1))).dropna()).mul(100) # multiply by 100 to express in %
returns_nif = ((np.log(nif) - np.log(nif.shift(1))).dropna()).mul(100) # multiply by 100 to express in %

#### Fit on complete dataaset

In [6]:
'''
S&P BSE 500
'''
model = arch_model(returns_snp, p = 1, q = 1, vol = 'GARCH', dist = 't')
# fit model
model_fit = model.fit(disp='off')
# forecast the test set
yhat = model_fit.forecast(horizon=1)
fulldataset_snp = round(np.sqrt(yhat.variance.values[-1][0])/100,4)
print("Standard Deviation of S&P BSE 500 using GARCH(1,1) fitted on the complete dataset : ",fulldataset_snp)

'''
NIFTY
'''
model = arch_model(returns_nif, p = 1, q = 1, vol = 'GARCH', dist = 't')
# fit model
model_fit = model.fit(disp='off')
# forecast the test set
yhat = model_fit.forecast(horizon=1)
fulldataset_nif = round(np.sqrt(yhat.variance.values[-1][0])/100,4)
print("Standard Deviation of NIFTY using GARCH(1,1) fitted on the complete dataset : ",fulldataset_nif)



Standard Deviation of S&P BSE 500 using GARCH(1,1) fitted on the complete dataset :  0.0107
Standard Deviation of NIFTY using GARCH(1,1) fitted on the complete dataset :  0.0116


#### Fit using rolling window of length ~ 5 years

In [7]:
'''
S&P BSE 500
'''
model = arch_model(returns_snp, p = 1, q = 1, vol = 'GARCH', dist = 't')
index = returns_snp.index
start_loc = 0
end_loc = np.where(index> '2014-03-03')[0].min()
forecasts = {}
for i in range(len(index[end_loc-1:])):
    sys.stdout.flush()
    res = model.fit(first_obs=i, last_obs=i + end_loc, disp="off")
    temp = res.forecast(horizon=1, reindex=False).variance
    fcast = temp.iloc[0]
    forecasts[fcast.name] = fcast
print('S&P Done!')
rolling_window_snp = np.sqrt(pd.DataFrame(forecasts).T)
rolling_window_stdev_snp = rolling_window_snp['h.1'][-1]

'''
NIFTY
'''
model = arch_model(returns_nif, p = 1, q = 1, vol = 'GARCH', dist = 't')
index = returns_nif.index
start_loc = 0
end_loc = np.where(index> '2014-03-03')[0].min()
forecasts = {}
for i in range(len(index[end_loc-1:])):
    sys.stdout.flush()
    res = model.fit(first_obs=i, last_obs=i + end_loc, disp="off")
    temp = res.forecast(horizon=1, reindex=False).variance
    fcast = temp.iloc[0]
    forecasts[fcast.name] = fcast
print('NIFTY Done!')
rolling_window_nif = np.sqrt(pd.DataFrame(forecasts).T)
rolling_window_stdev_nif = rolling_window_nif['h.1'][-1]

S&P Done!
NIFTY Done!


In [9]:
rolling_window_stdev_snp/=100
rolling_window_stdev_nif/=100
rolling_window_stdev_snp = round(rolling_window_stdev_snp,4)
rolling_window_stdev_nif = round(rolling_window_stdev_nif,4)
print("Standard Deviation of S&P BSE 500 using GARCH(1,1) with rolling window : ",rolling_window_stdev_snp)
print("Standard Deviation of NIFTY using GARCH(1,1) with rolling window : ",rolling_window_stdev_nif)

Standard Deviation of S&P BSE 500 using GARCH(1,1) with rolling window :  0.0099
Standard Deviation of NIFTY using GARCH(1,1) with rolling window :  0.0108


#### 99% VaR using forecasted standard deviations

In [69]:
z = stats.norm.ppf(0.99)

'''
S&P
'''
fulldataset_var_snp = (z*fulldataset_snp)
rolling_window_var_snp = (z*rolling_window_stdev_snp)
print("VaR of S&P BSE 500 using Expanding Window GARCH(1,1) : ",current_asset_value_snp*fulldataset_var_snp)
print("VaR of S&P BSE 500 using Rolling Window GARCH(1,1) : ",current_asset_value_snp*rolling_window_var_snp)

'''
NIFTY
'''
fulldataset_var_nif = (z*fulldataset_nif)
rolling_window_var_nif = (z*rolling_window_stdev_nif)
print("VaR of NIFTY using Expanding Window GARCH(1,1) : ",current_asset_value_nif*fulldataset_var_nif)
print("VaR of NIFTY using Rolling Window GARCH(1,1) : ",current_asset_value_nif*rolling_window_var_nif)



VaR of S&P BSE 500 using Expanding Window GARCH(1,1) :  0.024891922252236996
VaR of S&P BSE 500 using Rolling Window GARCH(1,1) :  0.023030843953004327
VaR of NIFTY using Expanding Window GARCH(1,1) :  0.02698563533887375
VaR of NIFTY using Rolling Window GARCH(1,1) :  0.02512455703964108


In [70]:
df = pd.DataFrame(columns=['VaR Terminology','S&P BSE 500','NIFTY'])
df = df.append({'VaR Terminology':'VaR with Expanding Window GARCH','S&P BSE 500':current_asset_value_snp*fulldataset_var_snp,'NIFTY':current_asset_value_nif*fulldataset_var_nif}, ignore_index=True)
df = df.append({'VaR Terminology':'VaR with Rolling Window GARCH (window ~ 5 years) ','S&P BSE 500':current_asset_value_snp*rolling_window_var_snp,'NIFTY':current_asset_value_nif*rolling_window_var_nif}, ignore_index=True)

In [71]:
df['Relative (Nifty/ S&P)'] = df['NIFTY']/df['S&P BSE 500']
df['Remarks'] = 0
for i in range(df.shape[0]): 
    if df['Relative (Nifty/ S&P)'][i]>=1:
        df['Remarks'][i] = "NIFTY is riskier"
    else:
        df['Remarks'][i] = "S&P is riskier"

In [74]:
df.to_csv (r'/Users/nesarasr/Documents/Sem6/FRM/Termproject/Results_GARCH.csv',index = False,  header=True)

In [75]:
df

,VaR Terminology,S&P BSE 500,NIFTY,Relative (Nifty/ S&P),Remarks
0,VaR with Expanding Window GARCH,0.024892,0.026986,1.084112,NIFTY is riskier
1,VaR with Rolling Window GARCH (window ~ 5 years),0.023031,0.025125,1.090909,NIFTY is riskier
